In [3]:
import os
import torch
from pathlib import Path
from dotenv import load_dotenv
from unsloth import FastLanguageModel, get_chat_template
from transformers import TextStreamer

load_dotenv()
parent_dir = Path.cwd().parent.parent
os.environ["HF_HOME"] = str(parent_dir / "huggingface_cache")

def load_inference_model(model_path):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_path,
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    tokenizer = get_chat_template(tokenizer, chat_template = "qwen-3")
    FastLanguageModel.for_inference(model)
    return model, tokenizer

MODEL_PATH = "unsloth/Qwen3-8B-unsloth-bnb-4bit"
model, tokenizer = load_inference_model(MODEL_PATH)



==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Quadro RTX 8000. Num GPUs = 1. Max memory: 47.266 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def telecom_chat(model, tokenizer, question):
    system_prompt = "Bạn là chuyên gia Viễn thông cao cấp. Hãy phân tích câu hỏi trắc nghiệm và trả về trả lời một cách ngắn gọn. Ví dụ như: Đáp án đúng là: 1. (đáp án)"
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        enable_thinking=False
    ).to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    with torch.no_grad():
        model.generate(
            input_ids=inputs,
            streamer=streamer,
            max_new_tokens=1024,
            use_cache=True,
            temperature=0.3,
            repetition_penalty=1.1
        )

In [9]:
user_input = """
3GPP đề xuất số lượng thiết bị kết nối mà hệ thống mMTC có thể đáp ứng trên mỗi km2 là bao nhiêu?\n
Lựa chọn:\n
1. 10^3 thiết bị / km2\n
2. 10^5 thiết bị / km2\n
3. 10^4 thiết bị / km2\n
4. 10^6 thiết bị / km2\n
"""
telecom_chat(model, tokenizer, user_input)

Đáp án đúng là: 4. 10^6 thiết bị / km2
